<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# _*Working with Different Backends*_ 

In this tutorial, we will first describe the various [backends](#backends) available in QISKit and how to find out more information about each of them, then we will explore how QISKit allows you to compile and run the same quantum circuit on different backends with different topology. 

The latest version of this notebook is available on https://github.com/QISKit/qiskit-tutorial.

***
### Contributors
Jay Gambetta, Joe Hellmers, Anna Phan

## The Backends<a id='backends'></a>

At the core of QISKit is the ability to access different backends to run your quantum programs. These backends are real devices, online simulators, and local simulations. As we continue to increase the flexibility of QISKit we expect there to be many different backends. These will include new hardware with different number of qubits, connectivity, different gate sets, and simulators with different properties. 

Current devices can be found on the devices page of the IBM Q Experience

Current simulators:
* QASM simulators - these simulate a quantum circuit and predict the outcomes of a quantum experiment.
    * Online QASM simulator – this runs online and can be used for up to 20 qubits with conditionals. 
    * Online HPC QASM simulator – this runs online and can be used for up to 32 qubits without conditionals. 
    * Local QASM simulator - this runs on your local machine, and should only be used for less than 10 qubits. 
* Local unitary simulator – predicts the unitary of a quantum circuit without measurement and conditional operations.
* Local statevector simulator - predicts the statevector of a quantum circuit without measurement and conditional operations.

The three important parts of our backends are: a provider, a backend, and a job object. The provider allows local and remote providers to be registered. QISkit comes with the default local provider which is a collection of simulators and it is simple to register the remote IBMQ provider. Each provider will give you access to different backends, which allow you to submit jobs to be run on the providers backends.

NOTE THIS IS WORK IN PROGRESS WHILE WE PUT THE SCHEMAS INTO THE BACKENDS

In [1]:
import sys, time
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
except:
    qx_config = {
        "APItoken":"YOUR_TOKEN_HERE",
        "url":"https://quantumexperience.ng.bluemix.net/api"}

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError, QuantumJob
from qiskit import available_backends, execute, register, get_backend, compile

from qiskit.tools.visualization import plot_histogram
from pprint import pprint

### The Local and IBMQ providers

A provider must be registered and it must support three funtions: `available_backends()` and `get_backend()` and a method to register the provider.

```
available_backends() - list all the backends of the resistered backends
```
```
get_backend('name') - get a backend instance from a name 
```

Lets start with the  `available_backends()` command

In [3]:
available_backends()

['local_clifford_simulator',
 'local_statevector_simulator',
 'local_unitary_simulator',
 'local_qasm_simulator']

This list is compact and picks the best of the diffent types. To list all the backends not just alias you can use:

In [4]:
available_backends(compact=False)

['local_qasm_simulator_cpp',
 'local_qasm_simulator_py',
 'local_statevector_simulator_cpp',
 'local_statevector_simulator_py',
 'local_statevector_simulator_sympy',
 'local_unitary_simulator_sympy',
 'local_unitary_simulator_py',
 'local_clifford_simulator_cpp']

To addd the IBMQ provider you need to register it using

```
register(arg,...)
```

In [5]:
register(qx_config['APItoken'], qx_config['url'])
available_backends()

['local_clifford_simulator',
 'ibmqx_hpc_qasm_simulator',
 'ibmqx4',
 'ibmqx5',
 'ibmqx2',
 'local_unitary_simulator',
 'local_qasm_simulator',
 'local_statevector_simulator',
 'ibmqx_qasm_simulator']

The `available_backends` function can also use filters.

Filter: show remote backends only

In [6]:
available_backends({'local': False})

['ibmqx4',
 'ibmqx_hpc_qasm_simulator',
 'ibmqx5',
 'ibmqx2',
 'ibmqx_qasm_simulator']

Filter: show only local backends

In [7]:
available_backends({'local': True})

['local_clifford_simulator',
 'local_statevector_simulator',
 'local_unitary_simulator',
 'local_qasm_simulator']

Filer: show only simulators

In [8]:
available_backends({'simulator': True})

['local_clifford_simulator',
 'ibmqx_hpc_qasm_simulator',
 'local_unitary_simulator',
 'local_qasm_simulator',
 'local_statevector_simulator',
 'ibmqx_qasm_simulator']

Fileter: show only real devices

In [9]:
available_backends({'simulator': False})

['ibmqx2', 'ibmqx4', 'ibmqx5']

Filter: show both a remote backend and a simulator backend

In [10]:
available_backends({'local': False, 'simulator': True})

['ibmqx_hpc_qasm_simulator', 'ibmqx_qasm_simulator']

The second function `get_backend()` allows you to make a backend instance

In [11]:
backends = []
for backend in available_backends():
    backends.append(get_backend(backend))
backends

### The Backends

The next part is the backend instances. These backends must support the following functions:

```
configuration - gets the configuration of the backend.
```
```
properties -  gets the properties of the backend (calibrations and parameters are going to be combined).
```
```
schema -  returns the schema for the qobj object supported by this backend (not implemented yet).
```
```
status - gets the status of the backend.
```
```
jobs  -  returns a list of the job id’s (not implemented yet).
```
```
retrieve_job - gets the data in the job (not implemented yet).
```
```
run  - runs a qobj on the backend.
```


Lets start with the `status`:

    `available` lets us know that the backend is taking jobs

    `pending_jobs` lets you know how many jobs are in the queue

In [12]:
for backend in backends:
    pprint(backend.status)

{'available': True, 'name': 'local_clifford_simulator_cpp'}
{'available': True, 'name': 'ibmqx_hpc_qasm_simulator', 'pending_jobs': 0}
{'available': True, 'name': 'ibmqx4', 'pending_jobs': 0}
{'available': True, 'name': 'ibmqx5', 'pending_jobs': 4}
{'available': False, 'name': 'ibmqx2', 'pending_jobs': 330}
{'available': True, 'name': 'local_unitary_simulator_py'}
{'available': True, 'name': 'local_qasm_simulator_cpp'}
{'available': True, 'name': 'local_statevector_simulator_cpp'}
{'available': True, 'name': 'ibmqx_qasm_simulator', 'pending_jobs': 0}


The next is `configuration`

In [13]:
for backend in backends:
    pprint(backend.configuration)

{'basis_gates': 'cx,id,x,y,z,h,s,sdg,snapshot,wait,noise,save,load',
 'coupling_map': 'all-to-all',
 'description': 'A C++ Clifford simulator with approximate noise',
 'exe': '/Users/jaygambetta/miniconda3/envs/QISKitenv/lib/python3.6/site-packages/qiskit/backends/local/qasm_simulator_cpp',
 'local': True,
 'name': 'local_clifford_simulator_cpp',
 'simulator': True,
 'url': 'https://github.com/QISKit/qiskit-sdk-py/src/qasm-simulator-cpp'}
{'basis_gates': 'u1,u2,u3,cx,id',
 'coupling_map': 'all-to-all',
 'local': False,
 'n_qubits': 32,
 'name': 'ibmqx_hpc_qasm_simulator',
 'online_date': '2017-12-09T12:00:00.000Z',
 'simulator': True}
{'basis_gates': 'u1,u2,u3,cx,id',
 'chip_name': 'Raven',
 'coupling_map': [[1, 0], [2, 0], [2, 1], [3, 2], [3, 4], [4, 2]],
 'description': '5 qubit transmon bowtie chip 3',
 'gate_set': 'SU2+CNOT',
 'local': False,
 'n_qubits': 5,
 'name': 'ibmqx4',
 'online_date': '2017-09-18T00:00:00.000Z',
 'simulator': False,
 'url': 'https://ibm.biz/qiskit-ibmqx4',


In [14]:
for backend in backends:
    pprint(backend.parameters)

{}
{}
{'backend': 'ibmqx4',
 'fridge_parameters': {'Temperature': {'date': '2018-05-13T01:59:20Z',
                                       'unit': 'K',
                                       'value': 0.021},
                       'cooldownDate': '2017-09-07'},
 'last_update_date': '2018-05-13T01:59:20.000Z',
 'qubits': [{'T1': {'date': '2018-05-13T01:59:20Z',
                    'unit': 'µs',
                    'value': 50.4},
             'T2': {'date': '2018-05-13T01:59:20Z',
                    'unit': 'µs',
                    'value': 12.4},
             'buffer': {'date': '2018-05-13T01:59:20Z',
                        'unit': 'ns',
                        'value': 10},
             'frequency': {'date': '2018-05-13T01:59:20Z',
                           'unit': 'GHz',
                           'value': 5.24229},
             'gateTime': {'date': '2018-05-13T01:59:20Z',
                          'unit': 'ns',
                          'value': 50},
             'name': 'Q0'},
 

{'backend': 'ibmqx2',
 'fridge_parameters': {'Temperature': {'date': '2018-04-13T15:53:48Z',
                                       'unit': 'K',
                                       'value': 0.0159},
                       'cooldownDate': '2017-05-07'},
 'last_update_date': '2018-04-13T15:53:48.000Z',
 'qubits': [{'T1': {'date': '2018-04-13T15:53:48Z',
                    'unit': 'µs',
                    'value': 62.4},
             'T2': {'date': '2018-04-13T15:53:48Z',
                    'unit': 'µs',
                    'value': 77.5},
             'buffer': {'date': '2018-04-13T15:53:48Z',
                        'unit': 'ns',
                        'value': 6.7},
             'frequency': {'date': '2018-04-13T15:53:48Z',
                           'unit': 'GHz',
                           'value': 5.27603},
             'gateTime': {'date': '2018-04-13T15:53:48Z',
                          'unit': 'ns',
                          'value': 83.3},
             'name': 'Q0'},
   

In [15]:
for backend in backends:
    pprint(backend.calibration)

{}
{}
{'backend': 'ibmqx4',
 'last_update_date': '2018-05-13T01:59:20.000Z',
 'multi_qubit_gates': [{'gateError': {'date': '2018-05-13T01:59:20Z',
                                      'value': 0.025213102663840553},
                        'name': 'CX1_0',
                        'qubits': [1, 0],
                        'type': 'CX'},
                       {'gateError': {'date': '2018-05-13T01:59:20Z',
                                      'value': 0.034465714682663684},
                        'name': 'CX2_0',
                        'qubits': [2, 0],
                        'type': 'CX'},
                       {'gateError': {'date': '2018-05-13T01:59:20Z',
                                      'value': 0.028811838355224106},
                        'name': 'CX2_1',
                        'qubits': [2, 1],
                        'type': 'CX'},
                       {'gateError': {'date': '2018-05-13T01:59:20Z',
                                      'value': 0.11260554765472736}

{'backend': 'ibmqx2',
 'last_update_date': '2018-04-13T15:53:48.000Z',
 'multi_qubit_gates': [{'gateError': {'date': '2018-04-13T15:53:48Z',
                                      'value': 0.02724902259050041},
                        'name': 'CX0_1',
                        'qubits': [0, 1],
                        'type': 'CX'},
                       {'gateError': {'date': '2018-04-13T15:53:48Z',
                                      'value': 0.041787588869506564},
                        'name': 'CX0_2',
                        'qubits': [0, 2],
                        'type': 'CX'},
                       {'gateError': {'date': '2018-04-13T15:53:48Z',
                                      'value': 0.03767722967662601},
                        'name': 'CX1_2',
                        'qubits': [1, 2],
                        'type': 'CX'},
                       {'gateError': {'date': '2018-04-13T15:53:48Z',
                                      'value': 0.039730922592824625},
     

### The Jobs object

To get a job first we must get a qobj which is a quantum object. This is a list of circuits compiled to a backend. To get this we have made a `compile` funtion that converts an array of circuits into a `qobj`. The reason this is an array is for current hardware lists of circuits are loaded onto the equipment at once and it is much more efficent to run this in a batch mode. The methods of the jobs object are:

```
status -- returns the status.
```
```
done - returns a bool true if done.
```
```
job_id  - gets the job_id (not supported in the local provider yet).
```
```
cancel - cancels the job (not supported on remote-public or local. Only supported on IBM Q premium devices).
```
```
result - gets the results from the circuit run.
```

In [16]:
my_backend = get_backend('local_qasm_simulator')

In [17]:
qr = QuantumRegister(3)
cr = ClassicalRegister(3)
circuit = QuantumCircuit(qr, cr)
circuit.x(qr[0])
circuit.x(qr[1])
circuit.ccx(qr[0], qr[1], qr[2])
circuit.cx(qr[0], qr[1])
circuit.measure(qr, cr)

qobj = compile(circuit, backend=my_backend)

In [18]:
job = my_backend.run(QuantumJob(qobj, backend=my_backend, preformatted=True))

In [19]:
job.status

{'status': <JobStatus.DONE: 'job has successfully run'>, 'status_msg': None}

In [20]:
job.done

True

In [21]:
job.cancel

<bound method LocalJob.cancel of <qiskit.backends.local.localjob.LocalJob object at 0x150b6b8550>>

In [22]:
result = job.result()
result.get_counts(circuit)

{'101': 1024}

In [23]:
my_backend = get_backend('ibmqx4')

In [24]:
qobj = compile(circuit, backend=my_backend)
job = my_backend.run(QuantumJob(qobj, backend=my_backend, preformatted=True))

In [27]:
job.status

{'job_id': 'c5ba50fe3eca8a664ed73beae34466eb',
 'status': <JobStatus.RUNNING: 'job is actively running'>,
 'status_msg': None}

In [28]:
job.done

False

In [29]:
job.cancel

<bound method IBMQJob.cancel of <qiskit.backends.ibmq.ibmqjob.IBMQJob object at 0x150b738dd8>>

In [30]:
job.status

{'job_id': 'c5ba50fe3eca8a664ed73beae34466eb',
 'status': <JobStatus.RUNNING: 'job is actively running'>,
 'status_msg': None}

In [31]:
qobj = compile(circuit, backend=my_backend)
job = my_backend.run(QuantumJob(qobj, backend=my_backend, preformatted=True))

In [32]:
lapse = 0
interval = 10
while not job.done:
    print('Status @ {} seconds'.format(interval * lapse))
    print(job.status)
    time.sleep(interval)
    lapse += 1
print(job.status)

Status @ 0 seconds
{'job_id': None, 'status': <JobStatus.INITIALIZING: 'job is being initialized'>, 'status_msg': 'job is begin initialized please wait a moment'}
Status @ 10 seconds
{'job_id': 'f0d4f260a1706aab8bc9ffb4ce453b00', 'queue_position': 3, 'status': <JobStatus.QUEUED: 'job is queued'>, 'status_msg': None}
Status @ 20 seconds
{'job_id': 'f0d4f260a1706aab8bc9ffb4ce453b00', 'status': <JobStatus.RUNNING: 'job is actively running'>, 'status_msg': None}
Status @ 30 seconds
{'job_id': 'f0d4f260a1706aab8bc9ffb4ce453b00', 'status': <JobStatus.RUNNING: 'job is actively running'>, 'status_msg': None}
{'job_id': 'f0d4f260a1706aab8bc9ffb4ce453b00', 'status': <JobStatus.DONE: 'job has successfully run'>, 'status_msg': None}


In [33]:
job.job_id

'f0d4f260a1706aab8bc9ffb4ce453b00'

In [34]:
result = job.result()
result.get_counts(circuit)

{'000': 29,
 '001': 156,
 '010': 61,
 '011': 10,
 '100': 92,
 '101': 608,
 '110': 40,
 '111': 28}